# Compute theoretical expected hotspot propensity

## Idea: compute expected number of hotspots in a region

### Intro

We model each trinucleotide context as an indepedendent region with constant mutation rate.

Given a signature's trinucleotide profile, and a trinucleotide content and a mutation rate in a region of interest, we can compute the hotspot rate for each of the 32 trinucleotide contexts in the region.

### Set-up

Consider a stack of $N$ copies of the same DNA region of length $L$ where all positions in the region have the same reference triplet and mutation rate.

Each position has identical mutability profile and each position can mutate in three different ways, non-necessarily with same mutation rate. 

The three posible substitutions are referred to as either $\{A, B, C\}$.

#### Per position mutation probability

So we will assume that each of the 3 specific mutation types $\{A, B, C\}$ has its own probability $p_A, p_B, p_C$.

#### Per position hotspot probability

A hotspot is called if the same mutation is found at the same position in at least two samples.

We can think of a position as a stack of $N$ letters picked from the dictionary $\Omega = \{R,A,B,C\}$. "R" means reference, it represents the event that there is not mutation at this position for that sample. Each position at each sample undergoes mutation with the same probability $p$ independently from other positions and samples. Thus for each level of the stack $R$ has probability $1-p$ and each $\{A,B,C\}$ have probabilities $p_A$, $p_B$, $p_C$, respectively, satisfying

$$p_A + p_B + p_C = p.$$ 

Given a position, what are the chances that the stack has at least two identical letters of the set $\{A, B, C\}$? Such an event will be a "hotspot".

$$
P = 1 
- \textrm{prob that stack has all R's} \\ 
- \textrm{prob that stack has exactly one A, B or C and rest R's} \\ 
- \textrm{prob that stack has two of non-repeated A, B or C and rest R's} \\
- \textrm{prob that stack has three of non-repeated A, B or C and rest R's}
$$

Or equivalently:
$$
% P = 1 - (1 - p)^N \\
% - N\cdot p_A\cdot (1-p)^{N-1}
% - N\cdot p_B\cdot (1-p)^{N-1}
% - N\cdot p_C\cdot (1-p)^{N-1} \\
% - \binom{N}{2}\cdot 2! \cdot p_A \cdot p_B \cdot (1-p)^{N-2} 
% - \binom{N}{2}\cdot 2! \cdot p_A \cdot p_C \cdot (1-p)^{N-2}
% - \binom{N}{2}\cdot 2! \cdot p_B \cdot p_C \cdot (1-p)^{N-2} \\
% - \binom{N}{3}\cdot 3! \cdot p_A \cdot p_B \cdot p_C \cdot (1-p)^{N-3}
$$

$$
P = 1 
- (1 - p)^N \\
- N\cdot (1-p)^{N-1} \cdot \left[p_A+p_B+p_C \right] \\
- N\cdot (N-1) \cdot (1-p)^{N-2} \cdot \left[ p_A \cdot p_B + p_A \cdot p_C + p_B \cdot p_C\right] \\
- N\cdot (N-1) \cdot (N-2) \cdot (1-p)^{N-3} \cdot p_A \cdot p_B \cdot p_C
$$

#### Hotspot propensity estimates

The number of hotspots follows a binomial distribution $\textrm{Binom}(L, P)$, i.e.

$$\textrm{Prob}(n) = \binom{L}{n} P^n (1 - P)^{L-n}$$

and the expected number of hotspots (expected hotspot propensity) is simply:

$$
e = LP
$$

### Imports

In [1]:
import os
import glob
import itertools
import json

from tqdm import tqdm
import pandas as pd
import numpy as np

from utils import triplets, mut_key_gen, mut_key_gen_cosmic, sum_dict, sbs_normalize, sbs_format

# Input Data

In [2]:
main_dir = ''

### Load COSMIC Signatures

In [3]:
cb = dict(zip('ACGT','TGCA'))

In [4]:
cosmic_df = pd.read_csv('./COSMIC_v3.2_SBS_GRCh38.txt', sep='\t', index_col='Type')
canonical_context_sorting = list(mut_key_gen_cosmic())
cosmic_df = cosmic_df.loc[canonical_context_sorting]

In [5]:
cosmic_df.head()

,SBS1,SBS2,SBS3,SBS4,SBS5,SBS6,SBS7a,SBS7b,SBS7c,SBS7d,...,SBS85,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94
Type,,,,,,,,,,,,,,,,,,,,,
A[C>A]A,0.000876,5.790059e-07,0.020920,0.042451,0.012052,0.000425,0.000067,0.002344,0.004841,0.000040,...,0.006108,0.002968,0.008946,0.000000,0.032297,0.002222,0.002934,0.011396,0.011628,0.015677
A[C>A]C,0.002220,1.455045e-04,0.016343,0.032990,0.009337,0.000516,0.000177,0.000457,0.001135,0.000754,...,0.000871,0.003735,0.004490,0.000000,0.017495,0.000704,0.052013,0.009653,0.008011,0.024523
A[C>A]G,0.000180,5.361861e-05,0.001808,0.016116,0.001908,0.000053,0.000073,0.000192,0.000388,0.000257,...,0.000316,0.000398,0.006357,0.000000,0.009971,0.000144,0.000209,0.004851,0.001817,0.001627
A[C>A]T,0.001265,9.759122e-05,0.012265,0.029663,0.006636,0.000180,0.000249,0.000714,0.001964,0.004051,...,0.002728,0.003639,0.004941,0.001738,0.020818,0.001771,0.000130,0.007800,0.008457,0.011141
C[C>A]A,0.000305,2.053143e-04,0.022376,0.080269,0.007379,0.001800,0.000451,0.001134,0.000108,0.014348,...,0.004236,0.006076,0.008436,0.000000,0.024492,0.001183,0.008071,0.018450,0.006456,0.079522


### Chunksize selector

In [6]:
chunksize = '1000kb'    # Replace with '500kb', '250kb', '100kb', '50kb', '25kb', '10kb'

### Load trinucleotide content

In [7]:
available_bins_f = f'{main_dir}/genomic_bins/data/hg38_{chunksize}_bin.nodrivers.filtered.mappable_positions.autosomes.binids.txt'
available_bins_df = pd.read_csv(available_bins_f, sep='\t', header=0)
available_bins = set(available_bins_df['BINID'].unique())
len(available_bins)

2196

In [8]:
triplet_counts_per_bin_fn = f'{main_dir}/genomic_bins/data/hg38_{chunksize}_bin.nodrivers.filtered.mappable_positions.autosomes.trinuc_per_bin.json'
with open(triplet_counts_per_bin_fn, 'rt') as f:
    triplet_counts_per_bin = json.load(f)

In [9]:
# load triplet abundance per chunk
triplet_abundance_array_dict = {}
for chunk, d in triplet_counts_per_bin.items():
    if chunk in available_bins: 
        triplet_abundance_array_dict[chunk] = sbs_format([d[t] for t in triplets()])

In [10]:
# genome-wide triplet abundance
triplet_abundance_array_genomewide = {}
for t, v in triplet_counts_per_bin.items():
    if t in available_bins: 
        triplet_abundance_array_genomewide = sum_dict(triplet_abundance_array_genomewide, v)
triplet_abundance_array_genomewide = sbs_format([triplet_abundance_array_genomewide[t] for t in triplets()])

### Normalized mutation rate per chunk

For each signature and tumor type, each chunk is given a weight and all the weights add up to 1.

In [11]:
zip_list = []
for fn in glob.glob(f'{main_dir}/genomic_bins/data/mutations_per_bin/*_*.{chunksize}.nodrivers.normmuts.total_maxprob.mutations_per_bin.relat_pcount.json'):
    cancer_type = os.path.basename(fn).split('SBS')[0][:-1]
    signature = 'SBS' + os.path.basename(fn).split('SBS')[1].split('.')[0]
    zip_list.append((cancer_type, signature))
cancer_types, signatures = tuple(zip(*zip_list))

In [12]:
mutrate_dict = {}
for cancer_type, signature in zip(cancer_types, signatures):
    
    mutrate_fn = f'{main_dir}/genomic_bins/data/mutations_per_bin/{cancer_type}' \
                 f'_{signature}.{chunksize}.nodrivers.normmuts.total_maxprob.mutations_per_bin.relat_pcount.json'
    with open(mutrate_fn, 'rt') as f:
        mutrate_dict[(cancer_type, signature)] = json.load(f)

# TripletRegion class

This class implements the method to compute expected number of hotspots described above.

In [13]:
from decimal import *

class TripletRegion:
    
    getcontext().prec = 100  # using instances of decimal.Decimal() we can do arithmetic at an arbitrarily precision level
    
    def __init__(self, N, L, pa, pb, pc):
        
        """
        N: number of sample
        L: region length
        pa, pb, pc: per-base probabilities of either of 3 possible mutation types
        """
        
        self.N = Decimal(N)
        self.L = L
        self.pa = Decimal(pa)
        self.pb = Decimal(pb)
        self.pc = Decimal(pc)
        self.p_mutation = self.pa + self.pb + self.pc
        self.q = 1 - self.p_mutation
    
    @property
    def p_hotspot(self):
        t0 = self.q ** self.N
        t1 = Decimal(self.N) * self.q ** (self.N - 1) * (self.pa + self.pb + self.pc)
        t2 = Decimal(self.N) * Decimal(self.N - 1) * self.q ** (self.N - 2) * (self.pa * self.pb + self.pa * self.pc + self.pb * self.pc)
        t3 = Decimal(self.N) * Decimal(self.N - 1) * Decimal(self.N - 2) * self.q ** (self.N - 3) * self.pa * self.pb * self.pc
        return float(Decimal(1) - (t0 + t1 + t2 + t3))
    
    @property
    def expected_hotspots(self):
        return self.p_hotspot * self.L
    
    @property
    def binomial_distribution(self):
        rv = binom(self.L, self.p_hotspot)
        return rv

## Expected number of hotspots per region with variable mutation rate

In [14]:
def expected_hotspots_per_chunk(ttype, signature, muts_per_sample, N):
    
    """
    ttype: cancer type
    signature: COSMIC signature
    muts_per_sample: # mutations per sample 
    N: # samples
    """
    
    # get normalized signature profile
    sig = cosmic_df[signature].values
    norm_sig = sbs_normalize(sig, triplet_abundance_array_genomewide)
    
    # get relative mutation rates
    relative_mutation_rate_dict = mutrate_dict[(ttype, signature)]
    
    # sanity check
    assert(len(relative_mutation_rate_dict.keys()) == len(triplet_abundance_array_dict.keys()))
    
    # dictionary with hotspot rate functions for each chunk
    res = {}
    for chunk, abundance in triplet_abundance_array_dict.items():
        
        # total mutrate for the chunk
        L = int(sum(triplet_abundance_array_genomewide) / 3)
        chunk_mutrate = muts_per_sample * relative_mutation_rate_dict[chunk]
        
        # estimate relative mutation load for each triplet genome
        triplet_region_weights = {t: {'length': triplet_counts_per_bin[chunk][t], 'weights': []} for t in triplets()}

        for i, s in enumerate(mut_key_gen()):
            # mut_key_gen() yields s ~ (reference triplet, alternate)
            triplet_region_weights[s[0]]['weights'] += [norm_sig[i]]

        # mutrate
        # dict: triplet -> mutation burden that corresponds to the "triplet" region in the chunk per sample, 
        # given i) the relative mutation rate of the chunk and ii) the number of mutations per sample

        load = {t: sum(v['weights']) * v['length'] for t, v in triplet_region_weights.items()}
        total_load = sum([v for t, v in load.items()])
        mutrate = {t: (v / total_load) * chunk_mutrate for t, v in load.items()}
        
        # probabilities of for each mutation type within each triplet region
        probabilities = {}
        for t, v in triplet_region_weights.items():
            if v['length'] > 0: 
                perposition_mutrate = mutrate[t] / v['length']
                total_rate = sum(v['weights'])
                pa = (v['weights'][0] / total_rate) * perposition_mutrate
                pb = (v['weights'][1] / total_rate) * perposition_mutrate
                pc = (v['weights'][2] / total_rate) * perposition_mutrate
                probabilities[t] = (pa, pb, pc)
            else: 
                probabilities[t] = (0, 0, 0)
       
        res[chunk] = 0
        region = {}
        for t, v in triplet_region_weights.items():
            region[t] = TripletRegion(N, v['length'], *probabilities[t])
        for t, v in region.items():
            res[chunk] += v.expected_hotspots
            
    return res

### Run

In [15]:
N = 100  # number of samples
n_muts = 300 # number of mutations
cancer_type = 'COADREAD'
signature = 'SBS1'

results_d = expected_hotspots_per_chunk(cancer_type, signature, n_muts, N)
expected_hotspot_propensity_sig = sum(results_d.values())

In [16]:
output_f = f'{chunksize}_{cancer_type}_{signature}_{int(n_muts)}_{int(N)}.nodrivers.relat_pcount.json'
with open(output_f, 'wt') as f:
    json.dump(results_d, f)

# Expected number of hotspots assuming same mutation rate across the genome

### load trinucleotide content

In [17]:
chunksize = '1000kb'

In [18]:
available_bins_f = f'{main_dir}/genomic_bins/data/hg38_{chunksize}_bin.nodrivers.filtered.mappable_positions.autosomes.binids.txt'
available_bins_df = pd.read_csv(available_bins_f, sep='\t', header=0)
available_bins = set(available_bins_df['BINID'].unique())
len(available_bins)

2196

In [19]:
triplet_counts_per_bin_fn = f'{main_dir}/genomic_bins/data/hg38_{chunksize}_bin.nodrivers.filtered.mappable_positions.autosomes.trinuc_per_bin.json'
with open(triplet_counts_per_bin_fn, 'rt') as f:
    triplet_counts_per_bin = json.load(f)

In [20]:
# load triplet abundance per chunk
triplet_abundance_array_dict = {}
for chunk, d in triplet_counts_per_bin.items():
    if chunk in available_bins: 
        triplet_abundance_array_dict[chunk] = sbs_format([d[t] for t in triplets()])

In [21]:
# genome-wide triplet abundance
triplet_abundance_array_genomewide = {}
for t, v in triplet_counts_per_bin.items():
    if t in available_bins: 
        triplet_abundance_array_genomewide = sum_dict(triplet_abundance_array_genomewide, v)
triplet_abundance_array_genomewide = sbs_format([triplet_abundance_array_genomewide[t] for t in triplets()])

In [22]:
L = int(sum(triplet_abundance_array_genomewide) / 3)

In [23]:
def expected_hotspots_uniform_genome(signature, mutrate_per_megabase):
    
    """Returns a function that produces a result for a given input number of samples"""
        
    # triplet abundance
    
    abundance = triplet_abundance_array_genomewide
        
    # read COSMIC signatures table
    
    cosmic_df = pd.read_csv('./COSMIC_v3.2_SBS_GRCh38.txt', sep='\t', index_col='Type')
    canonical_context_sorting = list(mut_key_gen_cosmic())
    cosmic_df = cosmic_df.loc[canonical_context_sorting]
    
    # get normalized signature profile

    sig = cosmic_df[signature].values
    norm_sig = sig / np.array(abundance)
    norm_sig /= sum(norm_sig)

    # estimate relative mutation burden for each triplet genome

    triplet_genome_weights = {
        s[0]: {'length': abundance[i], 'weights': []} for i, s in enumerate(mut_key_gen())}

    for i, s in enumerate(mut_key_gen()):
        triplet_genome_weights[s[0]]['weights'] += [norm_sig[i]]
        
    # total genome length

    L = sum([v['length'] for t, v in triplet_genome_weights.items()])
    
    # mutrates
    
    total_mutrate = mutrate_per_megabase * L / 1e6
    load = {k: sum(v['weights']) * v['length'] for k, v in triplet_genome_weights.items()}
    total_load = sum([v for k, v in load.items()])
    mutrate = {k: (v / total_load) * total_mutrate for k, v in load.items()}
    
    # probabilities of mutation types within each triplet genome

    probabilities = {}
    for k, v in triplet_genome_weights.items():
        perposition_mutrate = mutrate[k] / v['length']
        total_rate = sum(v['weights'])
        pa = (v['weights'][0] / total_rate) * perposition_mutrate
        pb = (v['weights'][1] / total_rate) * perposition_mutrate
        pc = (v['weights'][2] / total_rate) * perposition_mutrate
        probabilities[k] = (pa, pb, pc)
    
    # function instance
    
    def func_expected(N):
        
        genome = {}
        for triplet, v in triplet_genome_weights.items():
            genome[triplet] = TripletRegion(N, v['length'], *probabilities[triplet])
        
        res = {}
        for k, v in genome.items():
            res[k] = {'length': triplet_genome_weights[k]['length'], 
                      'expected_hotspots': v.expected_hotspots, 
                      'p_hotspots': v.p_hotspot}
        return res
        
    return func_expected

### Run

In [24]:
# uniform model output based on mutations per sample
selected_signatures = set(tuple(zip(*zip_list))[1])
m_grid = [300, 600]
N_grid = list(map(int, np.linspace(100, 1000, num=19)))

results = []

for m in m_grid:
    mutation_rate_per_megabase = m / (L / 1e6)
    for sig in selected_signatures:
        func_expected = expected_hotspots_uniform_genome(sig, mutation_rate_per_megabase)
        for n in N_grid:
            res = func_expected(n)
            total_expected_hotspots = sum([v['expected_hotspots'] for v in res.values()])
            results.append((m, sig, n, total_expected_hotspots))

In [25]:
with open('expected_hotspots_genomewide_disallowed.json', 'wt') as f:
    json.dump(results, f)

In [26]:
selected_signatures = set(tuple(zip(*zip_list))[1])
m_grid = [0.1, 0.5, 1., 5., 10.]
N_grid = list(map(int, np.linspace(100, 1000, num=19)))

results = []

for mutation_rate_per_megabase in m_grid:
    for sig in selected_signatures:
        func_expected = expected_hotspots_uniform_genome(sig, mutation_rate_per_megabase)
        for n in N_grid:
            res = func_expected(n)
            total_expected_hotspots = sum([v['expected_hotspots'] for v in res.values()])
            results.append((mutation_rate_per_megabase, sig, n, total_expected_hotspots))

In [27]:
with open('expected_hotspots_genomewide_disallowed_mutspermb.json', 'wt') as f:
    json.dump(results, f)